## Obtain a list of medications and concept codes from SNOMED CT-AU with Australian Medicines Terminology (AMT)

This notebook demonstrates how to match the **descriptions** and **relationship** tables obtained from the Australian Digital Health Agency. The steps in this notebook is written as a single Python function in [`src/snomed_ct.py`](src/snomed_ct.py).

Tools used:
- Python Data Analysis Library (Pandas) ([link](https://pandas.pydata.org/))

Data used:
- SNOMED CT-AU with AMT ([link](https://www.digitalhealth.gov.au/healthcare-providers/product-releases/snomed-ct-au-with-australian-medicines-terminology-amt-march-2025-release))

New to SNOMED CT-AU? Check out CSIRO's Shrimp Browser ([link](https://ontoserver.csiro.au/shrimp/?concept=138875005&valueset=http://snomed.info/sct?fhir_vs&fhir=https://tx.ontoserver.csiro.au/fhir))

In [1]:
# This cell imports required packages
import pandas as pd


### Step 1: Load the descriptions table using as a Pandas DataFrame

In [2]:
descriptions_df = pd.read_csv(
    "ncts_sct_rf2/Full/Terminology/sct2_Description_Full-en-au_AU1000036_20250331.txt",
    delimiter="\t",
    index_col=0
)

print(descriptions_df.shape)
descriptions_df.head(5)


(4037485, 8)


,effectiveTime,active,moduleId,conceptId,languageCode,typeId,term,caseSignificanceId
id,,,,,,,,
101013,20020131,1,900000000000207008,126813005,en,900000000000013009,Neoplasm of anterior aspect of epiglottis,900000000000020002
101013,20170731,1,900000000000207008,126813005,en,900000000000013009,Neoplasm of anterior aspect of epiglottis,900000000000448009
102018,20020131,1,900000000000207008,126814004,en,900000000000013009,Neoplasm of junctional region of epiglottis,900000000000020002
102018,20170731,1,900000000000207008,126814004,en,900000000000013009,Neoplasm of junctional region of epiglottis,900000000000448009
103011,20020131,1,900000000000207008,126815003,en,900000000000013009,Neoplasm of lateral wall of oropharynx,900000000000020002


### Step 2: Load the relationships table as a Pandas DataFrame

In [3]:
relationships_df = pd.read_csv(
    "ncts_sct_rf2/Full/Terminology/sct2_Relationship_Full_AU1000036_20250331.txt",
    delimiter="\t",
    index_col=0
)

print(relationships_df.shape)
relationships_df.head(5)


(8950677, 9)


,effectiveTime,active,moduleId,sourceId,destinationId,relationshipGroup,typeId,characteristicTypeId,modifierId
id,,,,,,,,,
100022,20020131,1,900000000000207008,100000000,102272007,0,116680003,900000000000011006,900000000000451002
100022,20090731,0,900000000000207008,100000000,102272007,0,116680003,900000000000011006,900000000000451002
101021,20020131,1,900000000000207008,10000006,29857009,0,116680003,900000000000011006,900000000000451002
102025,20020131,1,900000000000207008,10000006,9972008,0,116680003,900000000000011006,900000000000451002
103024,20020131,1,900000000000207008,1000004,19130008,0,116680003,900000000000011006,900000000000451002


### Step 3: Join the description and relationship tables

This joins the two tables by linking:
- `conceptId` from **description**, and
- `sourceId` from **relationships**.



In [4]:
joined_df = pd.merge(
    left=descriptions_df,
    right=relationships_df,
    left_on="conceptId",
    right_on="sourceId",
    suffixes=["_desc", "_rel"]
)

print(joined_df.shape)
joined_df.head(5)


(56776899, 17)


,effectiveTime_desc,active_desc,moduleId_desc,conceptId,languageCode,typeId_desc,term,caseSignificanceId,effectiveTime_rel,active_rel,moduleId_rel,sourceId,destinationId,relationshipGroup,typeId_rel,characteristicTypeId,modifierId
0,20020131,1,900000000000207008,126813005,en,900000000000013009,Neoplasm of anterior aspect of epiglottis,900000000000020002,20020131,1,900000000000207008,126813005,126699008,0,116680003,900000000000011006,900000000000451002
1,20020131,1,900000000000207008,126813005,en,900000000000013009,Neoplasm of anterior aspect of epiglottis,900000000000020002,20020131,1,900000000000207008,126813005,50410009,0,116680003,900000000000011006,900000000000451002
2,20020131,1,900000000000207008,126813005,en,900000000000013009,Neoplasm of anterior aspect of epiglottis,900000000000020002,20020731,0,900000000000207008,126813005,50410009,0,116680003,900000000000011006,900000000000451002
3,20020131,1,900000000000207008,126813005,en,900000000000013009,Neoplasm of anterior aspect of epiglottis,900000000000020002,20020131,1,900000000000207008,126813005,76712006,0,116680003,900000000000011006,900000000000451002
4,20020131,1,900000000000207008,126813005,en,900000000000013009,Neoplasm of anterior aspect of epiglottis,900000000000020002,20020731,0,900000000000207008,126813005,76712006,0,116680003,900000000000011006,900000000000451002


### Step 4: Filter the database

We filter the database and select:
- The full name (not synonym),
- Product name, and
- Active records.

In [5]:
joined_df = (
    joined_df.query(
        f"typeId_desc==900000000000003001" # Fully specified name (not synonym)
        # f" & destinationId==64572001" # Disease
        f" & destinationId==774167006" # Product name
        f" & active_desc==1"
    )
    .drop_duplicates(subset="conceptId")
    [["conceptId", "term"]] # Select only these two columns
)

print(joined_df.shape)
joined_df.head(10)


(13684, 2)


,conceptId,term
39007292,391000036103,Anastrozole (Chemmart) (trade product)
39007484,401000036100,Letrozole (Chemmart) (trade product)
39007634,421000036105,Quetiapine (Chemmart) (trade product)
39007696,431000036107,Anzole (trade product)
39007770,421000168103,Wart Removal System (Scholl) (trade product)
39007894,441000168109,Flixotide Junior Inhaler (trade product)
39007920,431000168100,Daktarin (trade product)
39008048,481000036106,Sequase (trade product)
39008090,461000168108,Adefin XL (trade product)
39008120,491000036108,Quetiapine (Terry White Chemists) (trade product)
